# Gaussian volume denoising

In [ ]:
use_OpenCV = True
local_debug = True

In [ ]:
import logging
import numpy as np
import scipy.ndimage
import matplotlib.pyplot as plt
#from ipywidgets import *
#import cv2
import time
#import kernels
#import skimage
#from skimage import io as skimage_io

In [ ]:
import logging
logging.basicConfig(format="[%(filename)s:%(lineno)s %(funcName)s()] %(message)s")
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

In [ ]:
if local_debug:
    !ln -sf ../../information_theory/src/information_theory/ .
else:
    !pip install "information_theory @ git+https://github.com/vicente-gonzalez-ruiz/information_theory"
import information_theory  # pip install "information_theory @ git+https://github.com/vicente-gonzalez-ruiz/information_theory"

In [ ]:
if use_OpenCV:
    from denoising.volume.gaussian import Monochrome_Denoising as gaussian_denoising

In [ ]:
import skimage.io

In [ ]:
from collections import namedtuple
Args = namedtuple("args", ["input", "output"])
args = Args("Corbel2301_block2_June2019_crop_ali_crop.tif", "denoised")

In [ ]:
%%bash -s "$args.input"
set -x
OUTPUT_FILENAME=$1
#rm -f $OUTPUT_FILENAME
if test ! -f $OUTPUT_FILENAME ; then
    FILEID="1Uqa6ywi8bllhyHxrODD5yjuesUkNO3O0"
    #wget --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O $OUTPUT_FILENAME #2> /dev/null
    #wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=FILEID' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=FILEID" -O FILENAME && rm -rf /tmp/cookies.txt
    gdown https://drive.google.com/uc?id=$FILEID
fi
set +x
# https://drive.google.com/file/d/1Uqa6ywi8bllhyHxrODD5yjuesUkNO3O0/view?usp=sharing

In [ ]:
noisy = skimage.io.imread(args.input, plugin="tifffile").astype(np.float32)

In [ ]:
Z_dim = noisy.shape[0]
Z2 = Z_dim//2

In [ ]:
plt.imshow(noisy[Z2], cmap="gray")

In [ ]:
def get_gaussian_kernel(sigma=1):
    number_of_coeffs = 3
    number_of_zeros = 0
    while number_of_zeros < 2 :
        delta = np.zeros(number_of_coeffs)
        delta[delta.size//2] = 1
        coeffs = scipy.ndimage.gaussian_filter1d(delta, sigma=sigma)
        number_of_zeros = coeffs.size - np.count_nonzero(coeffs)
        number_of_coeffs += 1
    return coeffs[1:-1]

sigma = np.array([1.5, 1.5, 1.5])
kernel = [None]*3
kernel[0] = get_gaussian_kernel(sigma[0])
kernel[1] = get_gaussian_kernel(sigma[1])
kernel[2] = get_gaussian_kernel(sigma[2])

In [ ]:
denoiser = gaussian_denoising(logger)
denoised = denoiser.filter(noisy, kernel)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
axes[0].imshow(denoised[Z2], cmap="gray")
axes[1].imshow(noisy[Z2], cmap="gray")
plt.show()